In [1]:
from openvino.inference_engine import IENetwork, IEPlugin
import numpy as np
import yaml
import os

In [2]:
os.chdir("/home/user/predictor")
os.listdir()

['notebooks',
 'sequential_1.xml',
 'sequential_1.bin',
 'input.txt',
 'sequential_1.mapping',
 'run.py',
 'run.sh',
 'test.txt',
 'docker-compose.yml',
 'Dockerfile']

In [3]:
classes=np.array(['Adipose Tissue', 'Blood', 'Brain', 'Breast', 'Lung', 'Pancreas',
       'Skin', 'Testis', 'Thyroid'])

y_test=np.genfromtxt("test.txt", dtype=int)

In [4]:
model_xml = "sequential_1.xml"
model_bin = "sequential_1.bin"

In [5]:
# Plugin initialization for specified device
plugin = IEPlugin(device="GPU")

RuntimeError: Cannot find plugin to use :Tried load plugin : clDNNPlugin,  error: Plugin clDNNPlugin cannot be loaded: cannot load plugin: clDNNPlugin from : clGetPlatformIDs error -1001, skipping
cannot load plugin: clDNNPlugin from /opt/intel/openvino_2019.3.376/python/python3.7/openvino/inference_engine: Cannot load library '/opt/intel/openvino_2019.3.376/python/python3.7/openvino/inference_engine/libclDNNPlugin.so': /opt/intel/openvino_2019.3.376/python/python3.7/openvino/inference_engine/libclDNNPlugin.so: cannot open shared object file: No such file or directory, skipping





In [20]:
net = IENetwork(model=model_xml, weights=model_bin)
exec_net = plugin.load(network=net)

RuntimeError: Can not init Myriad device: NC_ERROR

In [ ]:
print("Predicting using MYRIAD")
for i,data in enumerate(np.genfromtxt('input.txt')):
    res = exec_net.infer(inputs={'dense_1_input': data})
    #print(res['dense_1/Softmax'])
    print("real: %s \t predicted: %s"%(classes[y_test[i]],classes[np.argmax(res['dense_1/Softmax'])]))

In [18]:
print(yaml.dump(exec_net.requests[0].get_perf_counts(), default_flow_style=False))

dense_1/MatMul:
  cpu_time: 103
  exec_type: gemm_blas_FP32
  execution_index: 0
  layer_type: FullyConnected
  real_time: 103
  status: EXECUTED
dense_1/Softmax:
  cpu_time: 18
  exec_type: ref_any_FP32
  execution_index: 1
  layer_type: SoftMax
  real_time: 18
  status: EXECUTED
out_dense_1/Softmax:
  cpu_time: 11
  exec_type: unknown_FP32
  execution_index: 2
  layer_type: Output
  real_time: 11
  status: EXECUTED

